# River Channel Model

The `RiverChannel` class models a segment of a river using a linear reservoir model. In this model, the outflow from the river reach is directly proportional to the volume of water currently stored in it. This is a common and effective method for simulating flow routing in rivers. This notebook demonstrates the setup and simulation of a `RiverChannel`.

## State Variables

The state of the river channel is defined by:

- `volume` (float): The current volume of water stored in the river segment (m^3).
- `outflow` (float): The outflow from the end of the river segment (m^3/s).

## Parameters

The behavior of the river channel is controlled by a single parameter:

- `k` (float): The storage coefficient (s^-1), which determines the relationship between storage and outflow (`outflow = k * volume`). A smaller `k` results in more attenuation and lag.

## Simulation Example

The following example simulates a river channel responding to a flood wave (a pulse of high inflow). The simulation shows how the river channel attenuates (reduces the peak of) and lags (delays) the flood wave as it passes through. The inflow hydrograph and the resulting outflow hydrograph are plotted for comparison.

**Note:** To run this notebook, ensure you have `matplotlib` and `numpy` installed (`pip install matplotlib numpy`) and that you are running the Jupyter server from the root directory of the project.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.core.interfaces import State, Parameters

# Initial state and parameters for the river channel
initial_state = State(volume=10000.0, outflow=0.0)
parameters = Parameters(k=0.00002) # Storage coefficient

# Create a RiverChannel instance
river_channel = RiverChannel(name="main_river", initial_state=initial_state, parameters=parameters)

# Simulation settings
dt = 3600  # time step in seconds (1 hour)
simulation_duration = 3600 * 24 * 3  # 3 days
num_steps = int(simulation_duration / dt)

# Generate a flood wave inflow hydrograph
base_inflow = 10.0 # m^3/s
peak_inflow = 100.0 # m^3/s
inflow_hydrograph = base_inflow + (peak_inflow - base_inflow) * np.exp(-((np.arange(num_steps) - 24)**2) / 100))

# Store results for plotting
history = []

# Run the simulation loop
for i in range(num_steps):
    inflow = inflow_hydrograph[i]
    river_channel.set_inflow(inflow)
    current_state = river_channel.step(action=None, dt=dt)
    history.append(current_state.copy())

# Print the final state
print("Final State:", river_channel.get_state())

# Prepare data for plotting
time_in_hours = [i * dt / 3600 for i in range(num_steps)]
volumes = [s['volume'] for s in history]
outflows = [s['outflow'] for s in history]

# Plot the results
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.plot(time_in_hours, inflow_hydrograph, 'b--', label='Inflow')
ax1.plot(time_in_hours, outflows, 'r-', label='Outflow')
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('River Channel Flow Routing')
ax1.grid(True)
ax1.legend()

ax2 = ax1.twinx()
ax2.plot(time_in_hours, [v/1e6 for v in volumes], 'g:', label='Storage Volume')
ax2.set_ylabel('Storage (million m^3)', color='g')
ax2.tick_params(axis='y', labelcolor='g')
fig.tight_layout()
plt.show()
